# CAPSTONE - FINDINGS AND TECHNICAL REPORT WINE BLIND TASTING

I have buil a classification model that, based on a wine description given by an expert or semi-expert, is able to tell what country, variety and province was used to make that wine.

This dataset is the results of merging two datasets (big and small).


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns',500)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
df = pd.read_csv('wine_conca.csv')

In [3]:
df = df.dropna(axis=0, subset=['country'])

In [4]:
df.head()

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage
0,France,"Buxom and heady, this is a delightfully rich, ...",Vieilles Vignes,99,114.0,Châteauneuf-du-Pape,Rhône Valley,Anna Lee C. Iijima,Domaine de la Janasse 2016 Vieilles Vignes Red...,Rhône-style Red Blend,Domaine de la Janasse,2016.0
1,France,"Sultry and silken on the palate, this wine sta...",La Réserve,98,175.0,Châteauneuf-du-Pape,Rhône Valley,Anna Lee C. Iijima,Domaine le Clos du Caillou 2016 La Réserve Red...,Rhône-style Red Blend,Domaine le Clos du Caillou,2016.0
2,Portugal,The wine's fine perfumed black plum fruits giv...,NaN,98,120.0,Port,Port Blend,Roger Voss,Fonseca 2017 Port,Port,Fonseca,2017.0
3,France,"Veins of vanilla, smoke and toast amplify blac...",Hommage à Henry Tacussel,98,80.0,Châteauneuf-du-Pape,Rhône Valley,Anna Lee C. Iijima,Domaine Moulin-Tacussel 2016 Hommage à Henry T...,Grenache,Domaine Moulin-Tacussel,2016.0
4,France,"This juicy, fruit-forward wine drenches the pa...",La Muse,97,88.0,Châteauneuf-du-Pape,Rhône Valley,Anna Lee C. Iijima,Guillaume Gonnet 2016 La Muse Red (Châteauneuf...,Rhône-style Red Blend,Guillaume Gonnet,2016.0


In [5]:
print("There are {} types of grapes(varieties) in this dataset such as {}... \n".
      format(len(df.variety.unique()), ", ".join(df.variety.unique()[0:5])))
print("There are {} provinces producing wine in this dataset such as {}... \n".
      format(len(df.province.unique()), ", ".join(df.province.unique()[0:5])))
print("There are {} countries producing wine in this dataset such as {}... \n".
      format(len(df.country.unique()), ", ".join(df.country.unique()[0:5])))  
print("There are {} winery producing wine in this dataset such as {}... \n".
      format(len(df.winery.unique()), ", ".join(df.winery.unique()[0:5]))) 

There are 804 types of grapes(varieties) in this dataset such as Rhône-style Red Blend, Port, Grenache, Grenache-Mourvèdre, Champagne Blend... 

There are 1303 provinces producing wine in this dataset such as Châteauneuf-du-Pape, Port, Champagne, Trento, Santa Maria Valley... 

There are 44 countries producing wine in this dataset such as France, Portugal, Italy, US, Chile... 

There are 18582 winery producing wine in this dataset such as Domaine de la Janasse, Domaine le Clos du Caillou, Fonseca, Domaine Moulin-Tacussel, Guillaume Gonnet... 



In [6]:
print('The baselines of  my features are:')
print('The country baseline is', df.country.value_counts(normalize=True).max())
print('The variety baseline is', df.variety.value_counts(normalize=True).max())
print('The province baseline is', df.province.value_counts(normalize=True).max())

The baselines of  my features are:
The country baseline is 0.4260024592868068
The variety baseline is 0.1080593111556318
The province baseline is 0.23741115076627778


In [7]:
cvec = CountVectorizer(strip_accents='unicode',
                       stop_words="english", 
                       ngram_range=(1, 1))

X_all = cvec.fit_transform(df['description'])
columns = cvec.get_feature_names()
X_all

<166715x32129 sparse matrix of type '<class 'numpy.int64'>'
	with 4047438 stored elements in Compressed Sparse Row format>

###  THE VARIABLE COUNTRY AS A TARGET


In [8]:
# The target is the variable 'country'. This datset has 46 differentes countries.
y = df.country
X =  df.description

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### COUNT VECTORIZER  AND LOGISTIC REGRESSION 

In Countvectorizer:
- I am using stip_accents to Remove accents and perform other character normalization.
- I used 'stop_words' inside the CountVectorizer because I am going to eliminate most of the common words like 'and', 'the', 'of',..... that they are not relevant for determining wine language.
- the ngram_range is (1,1) because I am only want to extract 1 word.

In Logistic Regression:
- 'l1' I am using Lasso regularization where all the less important features will come 0.
- 'saga' this parameter handle with Lasso regulation and it's faster than 'liblinear'.
- 'ovr' in multiclass in the first performance. I change it to 'Multinomial' in the second performance because in the target there is more than two classes. And It does not improve the score.
- 'random_state' I am going to use the default because I am using 'saga' as a solver.

The best performer model was CountVectoricer and Logistic Regression with Ridge regularization.

I only calculated de Cross Validation score in the model that was the best performer. The score does not differ so much that the accurracy score.

All of the models have better scores than the baseline.

In [10]:
cvec = CountVectorizer(strip_accents='unicode', stop_words="english", ngram_range=(1, 1))

In [11]:
#CountVectorizer and Logistic Regresion with Ridge regularization
pipelines = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', verbose = 1, n_jobs = 2))
])
pipelines.fit(X_train, y_train)
predicted = pipelines.predict(X_test)
pipelines.score(X_test, y_test)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  44 out of  44 | elapsed:   28.6s finished


0.8731067990282818

In [12]:
print(classification_report(y_test, predicted))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                        precision    recall  f1-score   support

             Argentina       0.61      0.49      0.55       897
               Armenia       0.00      0.00      0.00         2
             Australia       0.77      0.65      0.70       641
               Austria       0.84      0.76      0.80       798
Bosnia and Herzegovina       0.00      0.00      0.00         2
                Brazil       1.00      0.08      0.14        13
              Bulgaria       0.80      0.43      0.56        46
                Canada       0.75      0.09      0.16        69
                 Chile       0.68      0.58      0.62      1144
                 China       0.00      0.00      0.00         1
               Croatia       0.71      0.24      0.36        21
        Czech Republic       1.00      0.29      0.44         7
               England       1.00      0.52      0.68        31
                France       0.84      0.90      0.87      5685
               Georgia       0.75      

In [13]:
# Cross Validation
cross = cross_val_score(pipelines, X_train, y_train, cv=5)
print(cross)
print(cross.mean())

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  43 out of  43 | elapsed:   23.0s finished
/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend Lo

[0.86905208 0.86908777 0.87026277 0.87100161 0.86884754]
0.8696503557261976


In [19]:
#CountVectorizer and Logistic Regresion with Ridge regularization
pipelines = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', verbose = 1, n_jobs = 2))
])
pipelines.fit(X_train, y_train)
predicted = pipelines.predict(X_test)
pipelines.score(X_test, y_test)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 out of  46 | elapsed:   29.0s finished


0.8715472512971238

In [21]:
#CountVectorizer and Logistic Regresion with Lasso regularization
pipeline1 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr', verbose = 1, n_jobs = 2))
])
pipeline1.fit(X_train, y_train)
predicted1 = pipeline1.predict(X_test)
pipeline1.score(X_test, y_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


max_iter reached after 18 seconds


/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


max_iter reached after 149 seconds
max_iter reached after 141 seconds
max_iter reached after 18 seconds
max_iter reached after 25 seconds
max_iter reached after 32 seconds
max_iter reached after 129 seconds
max_iter reached after 56 seconds
max_iter reached after 19 seconds
max_iter reached after 30 seconds
max_iter reached after 18 seconds
max_iter reached after 21 seconds
max_iter reached after 7736 seconds
max_iter reached after 7865 seconds
max_iter reached after 35 seconds
max_iter reached after 103 seconds
max_iter reached after 41 seconds
max_iter reached after 32 seconds
max_iter reached after 20 seconds
max_iter reached after 291 seconds
max_iter reached after 59 seconds
max_iter reached after 19 seconds
max_iter reached after 24 seconds
max_iter reached after 19 seconds
max_iter reached after 18 seconds
max_iter reached after 28 seconds
max_iter reached after 155 seconds
max_iter reached after 28 seconds
max_iter reached after 22 seconds
max_iter reached after 32 seconds
max_

[Parallel(n_jobs=2)]: Done  46 out of  46 | elapsed: 154.5min finished


0.8703775904987554

### TFIDFVECTORIZER  AND LOGISTIC REGRESSION 

The best model performer was when I included the Lasso regularization with all the features. 

All of the models have better scores than the baseline.

In [23]:
vect = TfidfVectorizer(stop_words='english', sublinear_tf=True, ngram_range = (1,1), strip_accents='unicode')

In [24]:
#TfidfVectorizer and Logistic Regression with Lasso regularization 
pipeline3 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr',verbose = 1, n_jobs = 2))
])
pipeline3.fit(X_train, y_train)
predicted3 = pipeline3.predict(X_test)
pipeline3.score(X_test, y_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


convergence after 13 epochs took 1 seconds
max_iter reached after 42 seconds


/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


max_iter reached after 48 seconds
max_iter reached after 8 seconds
max_iter reached after 10 seconds
max_iter reached after 12 seconds
max_iter reached after 42 seconds
max_iter reached after 13 seconds
max_iter reached after 9 seconds
max_iter reached after 9 seconds
max_iter reached after 8 seconds
max_iter reached after 8 seconds
max_iter reached after 50 seconds
max_iter reached after 12 seconds
max_iter reached after 11 seconds
max_iter reached after 32 seconds
max_iter reached after 17 seconds
max_iter reached after 11 seconds
max_iter reached after 9 seconds
max_iter reached after 87 seconds
max_iter reached after 19 seconds
max_iter reached after 8 seconds
max_iter reached after 9 seconds
max_iter reached after 9 seconds
max_iter reached after 8 seconds
max_iter reached after 52 seconds
max_iter reached after 9 seconds
max_iter reached after 12 seconds
max_iter reached after 9 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
convergence after 16 epoch

[Parallel(n_jobs=2)]: Done  46 out of  46 | elapsed:  8.7min finished


0.8612302432294635

In [26]:
# TFIDFVECTORIZER AND LOGISTIC REGRESSION inluding Ridge regularization solver lbfgs.
pipeline4 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(solver='lbfgs', multi_class='ovr', penalty = 'l2',verbose = 1, n_jobs = 2))
])
pipeline4.fit(X_train, y_train)
predicted4 = pipeline4.predict(X_test)
pipeline4.score(X_test, y_test)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 out of  46 | elapsed:   20.2s finished


0.856491617430945

### RANDOM FOREST CLASSIFIER, COUNTVECTORIZER AND TFDFVECTORIZER

TfidVectorizer performs a littel bit better than CountVectoriazer.

Logistic Regression and Naive Bayes performs better than Random Forest Classifier in this data set, with higher accuracy. Both of them have better scores than the baseline.

In [27]:
# Random Forest with CountVectorizer
cvecsr = CountVectorizer(strip_accents='unicode', stop_words="english", ngram_range=(1, 1),  max_features=1000)
pipelinesr = Pipeline([
    ('vect', cvecsr),
    #('tfidf', TfidfTransformer()),
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2))
])
pipelinesr.fit(X_train, y_train)
predictedsr = pipelinesr.predict(X_test)
pipelinesr.score(X_test, y_test)

0.7826530306211199

In [28]:
# Random Forest with TfidfVectorizer
pipelinesr = Pipeline([
    ('vect', vect),
    
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2))
])
pipelinesr.fit(X_train, y_train)
predictedsr = pipelinesr.predict(X_test)
pipelinesr.score(X_test, y_test)

0.7924601865459017

### NAIVE BAYES, COUNTVECTORIZER AND TFDFVECTORIZER

The best acurracy I got in MultinomialNB with CountVectorizer. MultinomialNB is one of the two classic Naive Bayes variants used in text classification. 

All of the models have better scores than the baseline.

In [29]:
cvect = CountVectorizer(lowercase=True, strip_accents='unicode', stop_words='english')

In [30]:
# MultinomialNB with TfidTransformer
pipelinesn = Pipeline([
    ('vect', cvect),
    ('tfidf', TfidfTransformer()),
    ('cls', MultinomialNB())
])
pipelinesn.fit(X_train, y_train)
predictedsn = pipelinesn.predict(X_test)
pipelinesn.score(X_test, y_test)

0.7351168161233242

In [31]:
# MultinomialNB with CountVectorizer
pipelinesn1 = Pipeline([
    ('vect', cvect),
    ('cls', MultinomialNB())
])
pipelinesn1.fit(X_train, y_train)
predictedsn1 = pipelinesn1.predict(X_test)
pipelinesn1.score(X_test, y_test)

0.8235911585640164

In [32]:
# BernoulliNB with BernoulliNb
pipelinesn2 = Pipeline([
    ('vect', cvect),
    ('cls', BernoulliNB())
])
pipelinesn2.fit(X_train, y_train)
predictedsn2 = pipelinesn2.predict(X_test)
pipelinesn2.score(X_test, y_test)

0.8220616021353807

In [33]:
vect1 = TfidfVectorizer(stop_words='english', sublinear_tf=True,
                             ngram_range = (1,1), strip_accents='unicode')

In [34]:
# MultinomialNB with  TdidfVectorizer
pipelinesn3 = Pipeline([
    ('vect',vect1) ,
    ('cls', MultinomialNB())
])
pipelinesn3.fit(X_train, y_train)
predictedsn3 = pipelinesn3.predict(X_test)
pipelinesn3.score(X_test, y_test)

0.7356566595687251

In [35]:
# BernoulliNB with  TdidfVectorizer
pipelinesn4 = Pipeline([
    ('vect', vect1),
    ('cls', BernoulliNB())
])
pipelinesn4.fit(X_train, y_train)
predictedsn4 = pipelinesn4.predict(X_test)
pipelinesn4.score(X_test, y_test)

0.8220616021353807

## REDUCTION COUNTRY AS A TARGET

I am going to train and test the models that got best accurancy in the models above.
I am reduccing the number of countries, removing the countries that appear less than 7 times. 

I am going to run the models that perform better above.

In [14]:
df1 = df[df['country'].map(df['country'].value_counts()) > 9]

In [15]:
df1.country.value_counts()

US                71021
France            27928
Italy             23002
Spain              8746
Portugal           7037
Chile              5408
Argentina          4595
Austria            3953
Australia          3355
Germany            3031
New Zealand        2044
Oregon             1757
South Africa       1514
Israel              720
Greece              555
Canada              311
Bulgaria            194
Uruguay             185
Hungary             171
Romania             158
England             133
Georgia             128
Moldova             116
Turkey               97
Croatia              94
Slovenia             81
Mexico               72
Brazil               56
Switzerland          38
Lebanon              36
Morocco              28
Luxembourg           20
India                17
Czech Republic       17
Peru                 17
Ukraine              14
Kosovo               12
Serbia               11
Macedonia            11
Cyprus               10
Name: country, dtype: int64

### VARIABLE COUNTRY AS A TARGET

In [16]:
y1 = df1.country
X1 =  df1.description
#Split the data and stratify it.
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, stratify = y1, random_state= 1)

In [17]:
len(df1.country.unique())

40

In [18]:
print('The variety baseline is', df1.variety.value_counts(normalize=True).max())

The variety baseline is 0.10807357281693182


### LOGISTIC REGRESSION WITH COUNTVECTORIZER AND TFIDFVECTORIZER 


The scores did not differ so much but the best performed model was with CountVectorizer and Logistic Regression with Ridge Regularization and multiclass ovr. I calculated the cross validation in this model and the score does not differ so much than the accuracy score.

All the models have better scores than the baseline.


In [19]:
cvec = CountVectorizer(strip_accents='unicode', stop_words="english", ngram_range=(1, 1))

In [20]:
# CountVetrorizer and LogisticRegression with Ridge regularization
pipelinesb2 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', verbose = 1, n_jobs = 2))
])
pipelinesb2.fit(X_train1, y_train1)
predictedsb2 = pipelinesb2.predict(X_test1)
pipelinesb2.score(X_test1, y_test1)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:   29.3s finished


0.8734815081436156

In [21]:
print(classification_report(y_test1, predictedsb2))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                precision    recall  f1-score   support

     Argentina       0.62      0.50      0.56       919
     Australia       0.77      0.64      0.70       671
       Austria       0.83      0.70      0.76       791
        Brazil       0.00      0.00      0.00        11
      Bulgaria       0.72      0.54      0.62        39
        Canada       1.00      0.08      0.15        62
         Chile       0.63      0.57      0.60      1082
       Croatia       0.60      0.16      0.25        19
        Cyprus       0.00      0.00      0.00         2
Czech Republic       1.00      0.33      0.50         3
       England       1.00      0.41      0.58        27
        France       0.84      0.90      0.87      5586
       Georgia       0.75      0.23      0.35        26
       Germany       0.79      0.75      0.77       606
        Greece       0.82      0.82      0.82       111
       Hungary       0.84      0.47      0.60        34
         India       0.00      0.00      0.00  

In [22]:
#Cross Validation
cross = cross_val_score(pipelinesb2, X_train1, y_train1, cv=5)
print(cross)
print(cross.mean())

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:   23.6s finished
/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:   24.1s finished
/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_cl

[0.86926219 0.87048769 0.87105362 0.86937934 0.86895517]
0.8698275991991622


In [24]:
# CountVetrorizer and LogisticRegression with Lasso regularization
pipelinesb1 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr', max_iter=100000, verbose = 1, n_jobs = 2))
])
pipelinesb1.fit(X_train1, y_train1)
predictedsb1 = pipelinesb1.predict(X_test1)
pipelinesb1.score(X_test1, y_test1)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


convergence after 1299 epochs took 1314 seconds
convergence after 1375 epochs took 1332 seconds
convergence after 982 epochs took 1153 seconds
convergence after 1270 epochs took 1318 seconds
convergence after 1473 epochs took 212 seconds
convergence after 588 epochs took 125 seconds
convergence after 1227 epochs took 455 seconds
convergence after 1410 epochs took 1313 seconds
convergence after 1061 epochs took 172 seconds
convergence after 1232 epochs took 121 seconds
convergence after 1615 epochs took 174 seconds
convergence after 946 epochs took 1132 seconds
convergence after 3389 epochs took 517 seconds
convergence after 1173 epochs took 204 seconds
convergence after 1146 epochs took 772 seconds
convergence after 2382 epochs took 590 seconds
convergence after 1306 epochs took 251 seconds
convergence after 2118 epochs took 234 seconds
convergence after 1236 epochs took 2642 seconds
convergence after 1515 epochs took 576 seconds
convergence after 4556 epochs took 445 seconds
convergen

[Parallel(n_jobs=2)]: Done  42 out of  42 | elapsed: 241.8min finished


0.8725516662167432

In [34]:
vect = TfidfVectorizer(stop_words='english', sublinear_tf=True,
                             ngram_range = (1,1), strip_accents='unicode')

In [38]:
# TfidfVectorizer and LogisticRegression with ridge regularization
pipelinesb3 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(solver='lbfgs', multi_class='multinomial', penalty = 'l2' ))
])
pipelinesb3.fit(X_train1, y_train1)
predictedsb3 = pipelinesb3.predict(X_test1)
pipelinesb3.score(X_test1, y_test1)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8614535528960077

In [39]:
# TfidfVectorizer and LogisticRegression with Lasso regularization
pipelinesb3 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(solver='saga', multi_class='ovr', penalty = 'l1' ))
])
pipelinesb3.fit(X_train1, y_train1)
predictedsb3 = pipelinesb3.predict(X_test1)
pipelinesb3.score(X_test1, y_test1)

0.8639731245688232

### RANDOM FOREST CLASSIFIER, COUNTVECTORIZER AND TFIDFVECTORIZER

Logistic Regression and Naive Bayes perform better than Random Forest Classifier in this data set, with higher accuracy.

The best performed model was Random Forest with Tfidfvectorizer.



In [40]:
# TfidfVectorizer and RandomForest
pipelinesb4 = Pipeline([
    ('vect', vect),
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2))
])
pipelinesb4.fit(X_train1, y_train1)
predictedsb4 = pipelinesb4.predict(X_test1)
pipelinesb4.score(X_test1, y_test1)

0.7911455052641051

### NAIVE BAYES, COUNTVECTORIZER AND TFIDFVECTORIZER

The best accuracy I got in MultinomialNB with CountVectorizer. MultinomialNB is one of the two classic Naive Bayes variants used in text classification.

All of the models have better scores than the baseline




In [45]:
# CountVectorizer with MultinomialNB
pipelinebn5 = Pipeline([
    ('cvect', cvec),
    ('cls', MultinomialNB())
])
pipelinebn5.fit(X_train1, y_train1)
predictedbn5 = pipelinebn5.predict(X_test1)
pipelinebn5.score(X_test1, y_test1)

0.8179909415399382

We can observe tat all the models show that the scores do not differ of the other models with all the countries as a traget.

##  VARIABLE VARIETY AS A TARGET

By looking at the varieties, we can find out that there are 805 varieties in this dataset. I am going to limit the number of varieties. I am going to define the varieties that appear more than 100 times. I am also going to replace some of the varieties that they are similar.

I am going to train some of the models I define above with this dataset.


In [5]:
df2 = df[df['variety'].map(df['variety'].value_counts()) > 100]

In [9]:
df2.variety = df2.variety.apply(lambda x: str(x).replace('Syrah', 'Shiraz').
                                    replace('Pinot Gris', 'Pinot Grigio').replace('Petite Sirah', 'Shiraz').
                                   replace('Grenache', 'Garnacha').replace('Garnacha Blanc', 'Garnacha').
                                   replace('Rosato', 'Rosé').replace('Pinot Bianco', 'Pinot Blanc').
                                   replace('Zinfandel', 'Primitivo').replace('Alvarinho', 'Alvariño').
                                   replace('Alvariño', 'Albariño').replace('Portuguese Sparkling', 'Sparkling').
                               replace('Garnachaa', 'Garnacha').replace('Pinot Nero', 'Pinot Noir').
                                replace('Red Blends', 'Red Blend'). replace('Muscat', 'Muscato').
                               replace('Red Blends', 'Red Blend'))

In [7]:
len(df2.variety.unique())

89

In [8]:
print('The variety baseline is', df2.variety.value_counts(normalize=True).max())

The variety baseline is 0.11543264263483834


### DEFINE TARGET AND SPLIT THE DATA

In [10]:
X2 = df2.description
y2 = df2.variety

In [11]:
#Split and stratify the data
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, stratify = y2, random_state = 1)

### LOGISTIC REGRESSION WITH COUNTVECTORIZER AND TFIDFVECTORIZER  
The scores did not differ so much but the best performed model was CountVectorizer and Logistic Regression with Ridge Regularization. I calculated the cross validation in this model and the score does not differ so much than the accuracy score.
 
All of the models have better scores than the baseline.
 


In [13]:
cvec = CountVectorizer(strip_accents='unicode', stop_words="english", ngram_range=(1, 1))

In [14]:
# CountVectorizer and Logistic Regression wiht Ridge regularization
pipelinesb6 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', verbose = 1, n_jobs = 2))
])
pipelinesb6.fit(X_train2, y_train2)
predictedsb6 = pipelinesb6.predict(X_test2)
pipelinesb6.score(X_test2, y_test2)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   34.0s
[Parallel(n_jobs=2)]: Done  89 out of  89 | elapsed:  1.1min finished


0.6645643837353025

In [15]:
print(classification_report(y_test2, predictedsb6))

                               precision    recall  f1-score   support

                    Aglianico       0.70      0.38      0.50        86
                     Albariño       0.70      0.47      0.56       172
                      Barbera       0.64      0.47      0.54       174
                Blaufränkisch       0.77      0.55      0.64        60
                      Bonarda       0.67      0.25      0.36        24
     Bordeaux-style Red Blend       0.68      0.77      0.72      1556
   Bordeaux-style White Blend       0.72      0.44      0.55       223
               Cabernet Franc       0.55      0.32      0.40       375
           Cabernet Sauvignon       0.58      0.70      0.64      2445
    Cabernet Sauvignon-Merlot       1.00      0.04      0.07        27
    Cabernet Sauvignon-Shiraz       0.50      0.04      0.08        23
                    Carmenère       0.51      0.37      0.43       149
              Champagne Blend       0.76      0.66      0.70       374
     

In [16]:
cross = cross_val_score(pipelinesb6, X_train2, y_train2, cv=5)
print(cross)
print(cross.mean())

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   28.7s
[Parallel(n_jobs=2)]: Done  89 out of  89 | elapsed:   53.9s finished
/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   26.6s
[Parallel(n_jobs=2)]: Done  89 out of  89 | elapsed:   51.9s finished
/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklear

[0.65147559 0.65903872 0.65911342 0.65678268 0.65288633]
0.6558593492037401


In [11]:
# CountVectorizer and Logistic Regression wiht Lasso regularization
pipelinesb5 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr', verbose = 1, n_jobs = 2))
])
pipelinesb5.fit(X_train2, y_train2)
predictedsb5 = pipelinesb5.predict(X_test2)
pipelinesb5.score(X_test2, y_test2)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


max_iter reached after 54 seconds


/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


max_iter reached after 80 seconds
max_iter reached after 51 seconds
max_iter reached after 77 seconds
max_iter reached after 33 seconds
max_iter reached after 70 seconds
max_iter reached after 195 seconds
max_iter reached after 124 seconds
max_iter reached after 41 seconds
max_iter reached after 39 seconds
max_iter reached after 64 seconds
max_iter reached after 255 seconds
max_iter reached after 93 seconds
max_iter reached after 68 seconds
max_iter reached after 49 seconds
max_iter reached after 37 seconds
max_iter reached after 33 seconds
max_iter reached after 225 seconds
max_iter reached after 40 seconds
max_iter reached after 29 seconds
max_iter reached after 51 seconds
max_iter reached after 65 seconds
max_iter reached after 34 seconds
max_iter reached after 38 seconds
max_iter reached after 75 seconds
max_iter reached after 124 seconds
max_iter reached after 30 seconds
max_iter reached after 42 seconds
max_iter reached after 32 seconds
max_iter reached after 31 seconds
max_iter 

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 26.5min


max_iter reached after 58 seconds
max_iter reached after 69 seconds
max_iter reached after 34 seconds
max_iter reached after 130 seconds
max_iter reached after 36 seconds
max_iter reached after 55 seconds
max_iter reached after 95 seconds
max_iter reached after 273 seconds
max_iter reached after 67 seconds
max_iter reached after 32 seconds
max_iter reached after 32 seconds
max_iter reached after 152 seconds
max_iter reached after 92 seconds
max_iter reached after 218 seconds
max_iter reached after 116 seconds
max_iter reached after 59 seconds
max_iter reached after 141 seconds
max_iter reached after 158 seconds
max_iter reached after 47 seconds
max_iter reached after 33 seconds
max_iter reached after 42 seconds
max_iter reached after 36 seconds
max_iter reached after 114 seconds
max_iter reached after 28 seconds
max_iter reached after 170 seconds
max_iter reached after 210 seconds
max_iter reached after 33 seconds
max_iter reached after 122 seconds
max_iter reached after 41 seconds
max

[Parallel(n_jobs=2)]: Done  91 out of  91 | elapsed: 56.5min finished


0.6561658162456818

In [17]:
print(classification_report(y_test2, predictedsb5))

                               precision    recall  f1-score   support

                    Aglianico       0.70      0.38      0.50        86
                     Albariño       0.70      0.47      0.56       172
                      Barbera       0.64      0.47      0.54       174
                Blaufränkisch       0.77      0.55      0.64        60
                      Bonarda       0.67      0.25      0.36        24
     Bordeaux-style Red Blend       0.68      0.77      0.72      1556
   Bordeaux-style White Blend       0.72      0.44      0.55       223
               Cabernet Franc       0.55      0.32      0.40       375
           Cabernet Sauvignon       0.58      0.70      0.64      2445
    Cabernet Sauvignon-Merlot       1.00      0.04      0.07        27
    Cabernet Sauvignon-Shiraz       0.50      0.04      0.08        23
                    Carmenère       0.51      0.37      0.43       149
              Champagne Blend       0.76      0.66      0.70       374
     

In [14]:
import joblib

In [16]:
joblib.dump(pipelinesb5, 'pipelinesb1_conca_scountrysb5')

['pipelinesb1_conca_scountrysb5']

In [17]:
pipe_reloaded = joblib.load('pipelinesb1_conca_scountrysb5')

In [19]:
vect = TfidfVectorizer(stop_words='english', sublinear_tf=True,
                             ngram_range = (1,1), strip_accents='unicode')

In [39]:
# TfidfVectorizer and Logistic Regression with Lasso regularization
cvec = CountVectorizer(strip_accents='unicode', stop_words="english", ngram_range=(1, 1))
pipeline11 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr'))
])
pipeline11.fit(X_train2, y_train2)
predicted11 = pipeline11.predict(X_test2)
pipeline11.score(X_test2, y_test2)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.6474503216809812

In [40]:
print(classification_report(y_test2, predicted11))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                               precision    recall  f1-score   support

                    Aglianico       0.89      0.29      0.44        86
                     Albariño       0.70      0.39      0.50       172
                      Barbera       0.80      0.44      0.57       174
                Blaufränkisch       0.67      0.33      0.44        60
                      Bonarda       0.80      0.33      0.47        24
     Bordeaux-style Red Blend       0.68      0.77      0.72      1556
   Bordeaux-style White Blend       0.67      0.38      0.48       223
               Cabernet Franc       0.64      0.29      0.40       375
           Cabernet Sauvignon       0.56      0.72      0.63      2445
    Cabernet Sauvignon-Merlot       0.00      0.00      0.00        27
    Cabernet Sauvignon-Shiraz       0.00      0.00      0.00        23
                    Carmenère       0.73      0.34      0.47       149
              Champagne Blend       0.77      0.61      0.68       374
     

In [20]:
# TfidfVectorizer and Logistic Regression with Ridge regularization
pipelinesb6 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', verbose = 1, n_jobs = 2))
])
pipelinesb6.fit(X_train2, y_train2)
predictedsb6 = pipelinesb6.predict(X_test2)
pipelinesb6.score(X_test2, y_test2)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   21.0s
[Parallel(n_jobs=2)]: Done  89 out of  89 | elapsed:   42.8s finished


0.6360092542705923

In [21]:
print(classification_report(y_test2, predictedsb6))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                               precision    recall  f1-score   support

                    Aglianico       1.00      0.20      0.33        86
                     Albariño       0.75      0.35      0.48       172
                      Barbera       0.90      0.38      0.53       174
                Blaufränkisch       0.75      0.25      0.38        60
                      Bonarda       0.00      0.00      0.00        24
     Bordeaux-style Red Blend       0.67      0.77      0.72      1556
   Bordeaux-style White Blend       0.65      0.28      0.39       223
               Cabernet Franc       0.70      0.23      0.34       375
           Cabernet Sauvignon       0.55      0.74      0.63      2445
    Cabernet Sauvignon-Merlot       0.00      0.00      0.00        27
    Cabernet Sauvignon-Shiraz       0.00      0.00      0.00        23
                    Carmenère       0.67      0.24      0.35       149
              Champagne Blend       0.79      0.59      0.67       374
     

### RANDOM FOREST CLASSIFIER, COUNTVECTORIZER AND TFIDFVECTORIZER
Logistic Regression performs better than Random Forest Classifier in this data set, with higher accuracy. The best performed model was Random Forest with CountVectorizer.

All of the models have better scores than the baseline


In [24]:
#RandomForestClassifier with CountVectoriazer
pipelinesb7 = Pipeline([
    ('vect', cvec),
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2))
])
pipelinesb7.fit(X_train2, y_train2)
predictedsb7 = pipelinesb7.predict(X_test2)
pipelinesb7.score(X_test2, y_test2)

0.644312743637689

In [25]:
print(classification_report(y_test2, predictedsb7))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                               precision    recall  f1-score   support

                    Aglianico       1.00      0.20      0.33        86
                     Albariño       0.98      0.36      0.53       172
                      Barbera       1.00      0.33      0.50       174
                Blaufränkisch       1.00      0.23      0.38        60
                      Bonarda       1.00      0.04      0.08        24
     Bordeaux-style Red Blend       0.59      0.83      0.69      1556
   Bordeaux-style White Blend       0.88      0.25      0.39       223
               Cabernet Franc       0.98      0.23      0.37       375
           Cabernet Sauvignon       0.50      0.81      0.62      2445
    Cabernet Sauvignon-Merlot       1.00      0.07      0.14        27
    Cabernet Sauvignon-Shiraz       1.00      0.09      0.16        23
                    Carmenère       1.00      0.28      0.43       149
              Champagne Blend       0.91      0.52      0.66       374
     

In [53]:
#RandomForestClassifier with TfidVectorizer
pipelinesb7 = Pipeline([
    ('vect', vect),
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2))
])
pipelinesb7.fit(X_train2, y_train2)
predictedsb7 = pipelinesb7.predict(X_test2)
pipelinesb7.score(X_test2, y_test2)

0.644946597787849

In [55]:
print(classification_report(y_test2, predictedsb7))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                               precision    recall  f1-score   support

                    Aglianico       1.00      0.20      0.33        86
                     Albariño       0.97      0.38      0.55       172
                      Barbera       1.00      0.37      0.54       174
                Blaufränkisch       0.93      0.23      0.37        60
                      Bonarda       1.00      0.04      0.08        24
     Bordeaux-style Red Blend       0.60      0.82      0.70      1556
   Bordeaux-style White Blend       0.89      0.28      0.42       223
               Cabernet Franc       0.99      0.21      0.35       375
           Cabernet Sauvignon       0.53      0.81      0.64      2445
    Cabernet Sauvignon-Merlot       1.00      0.07      0.14        27
    Cabernet Sauvignon-Shiraz       1.00      0.09      0.16        23
                    Carmenère       1.00      0.26      0.41       149
              Champagne Blend       0.89      0.54      0.67       374
     

### NAIVE BAYES, COUNTVECTORIZER AND TFIDFVECTORIZER

The best accuracy I got in MultinomialNB with CountVectorizer. MultinomialNB is one of the two classic Naive Bayes variants used in text classification.

Logistic Regression and some of the Random Forest Classifier models perform better than Naive Bayes in this data set, with higher accuracy.

All of the models have better scores than the baseline


In [60]:
# CountVectorizer with MultinomialNB
pipelinebn8 = Pipeline([
    ('cvect', cvec),
    ('cls', MultinomialNB())
])
pipelinebn8.fit(X_train2, y_train2)
predictedbn8 = pipelinebn8.predict(X_test2)
pipelinebn8.score(X_test2, y_test2)

0.5376350901657528

In [61]:
print(classification_report(y_test2, predictedbn8))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                               precision    recall  f1-score   support

                    Aglianico       0.00      0.00      0.00        86
                     Albariño       0.90      0.05      0.10       172
                      Barbera       1.00      0.02      0.03       174
                Blaufränkisch       0.00      0.00      0.00        60
                      Bonarda       0.00      0.00      0.00        24
     Bordeaux-style Red Blend       0.53      0.64      0.58      1556
   Bordeaux-style White Blend       0.78      0.17      0.28       223
               Cabernet Franc       0.59      0.03      0.05       375
           Cabernet Sauvignon       0.48      0.70      0.57      2445
    Cabernet Sauvignon-Merlot       0.00      0.00      0.00        27
    Cabernet Sauvignon-Shiraz       0.00      0.00      0.00        23
                    Carmenère       0.67      0.01      0.03       149
              Champagne Blend       0.65      0.59      0.62       374
     

In [22]:
# MultinomialNB with  TdidfVectorizer
pipelines12 = Pipeline([
    ('vect',vect) ,
    ('cls', MultinomialNB())
])
pipelines12.fit(X_train2, y_train2)
predictes12 = pipelines12.predict(X_test2)
pipelines12.score(X_test2, y_test2)

0.4202136088486039

In [23]:
print(classification_report(y_test2, predictes12))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                               precision    recall  f1-score   support

                    Aglianico       0.00      0.00      0.00        86
                     Albariño       0.00      0.00      0.00       172
                      Barbera       0.00      0.00      0.00       174
                Blaufränkisch       0.00      0.00      0.00        60
                      Bonarda       0.00      0.00      0.00        24
     Bordeaux-style Red Blend       0.48      0.57      0.52      1556
   Bordeaux-style White Blend       0.00      0.00      0.00       223
               Cabernet Franc       0.00      0.00      0.00       375
           Cabernet Sauvignon       0.37      0.70      0.48      2445
    Cabernet Sauvignon-Merlot       0.00      0.00      0.00        27
    Cabernet Sauvignon-Shiraz       0.00      0.00      0.00        23
                    Carmenère       0.00      0.00      0.00       149
              Champagne Blend       0.93      0.04      0.07       374
     

In [26]:
# BernoulliNB with CountVectorizer
pipelines9 = Pipeline([
    ('vect', cvec),
    ('cls', BernoulliNB())
])
pipelines9.fit(X_train2, y_train2)
predictes9 = pipelines9.predict(X_test2)
pipelines9.score(X_test2, y_test2)

0.5197920958387475

## PROVINCE AS A TARGET

By looking at the provinces, we can find out that there are 1303 provinces in this dataset. Each one having different number of inputs, many of them having only one. Certainly, the latter ones will not be of use, because we will not be able to train and test on one sample. I will keep the provinces thay appear more than 100 times. I will also reduce the number by joining some of them.

In [29]:
df3 = df[df['province'].map(df['province'].value_counts()) > 100]

In [30]:
df3.province = df3.province.apply(lambda x: str(x).replace('Blaye Côtes de Bordeaux', 'Bordeaux').
                                    replace('Bordeaux Rosé', 'Bordeaux').replace('Bordeaux Supérieur', 'Bordeaux').
                                   replace('Bordeaux Blanc', 'Bordeaux').replace('Sta. Rita Hills', 'California').
                                   replace('Cava', 'Penedes').replace('Mendocino County', 'California').
                                   replace('Russian River Valley', 'Sonoma Valley').replace('North Coast', 'Sonoma Valley').
                                 replace('Alentejano', 'Alentejo'))





/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [31]:
df3 = df3[df3.province!='Other']

In [32]:
(len(df3.province.unique()))

149

In [34]:
print('The province baseline is', df3.province.value_counts(normalize=True).max())

The province baseline is 0.2677460940891997


In [35]:
X3 = df3.description
y3 = df3.province

In [36]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, stratify = y3, random_state = 1)

### LOGISTIC REGRESSION WITH COUNTVECTORIZER AND TFIDFVECTORIZER  
The scores did not differ so much but the best performed model was with CountVectorizer and Logistic Regression with Ridge Regularization. I calculated the cross validation in this model and the score does not differ so much than the accuracy score.
 
All of the models have better scores than the baseline.


In [111]:
# CountVectorizer and Logisitic Regression with Ridge regularization
pipelinep1 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', multi_class='ovr', verbose = 1, n_jobs = 2))
])
pipelinep1.fit(X_train3, y_train3)
predictedp1 = pipelinep1.predict(X_test3)
pipelinep1.score(X_test3, y_test3)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   43.7s
[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed:  2.3min finished


0.6884643644379133

In [112]:
print(classification_report(y_test3, predictedp1))

                         precision    recall  f1-score   support

       Aconcagua Valley       0.27      0.12      0.16        26
      Adelaida District       0.05      0.04      0.04        26
               Alentejo       0.48      0.33      0.39       269
       Alexander Valley       0.00      0.00      0.00        24
                 Alsace       0.69      0.81      0.75       654
             Alto Adige       0.84      0.91      0.88        35
          Amador County       0.00      0.00      0.00        31
                America       1.00      0.17      0.29        29
              Andalucia       0.85      0.58      0.69        38
        Anderson Valley       0.12      0.06      0.09        31
        Australia Other       0.23      0.06      0.10        49
                  Baden       0.67      0.19      0.30        21
               Bairrada       0.83      0.41      0.55        37
             Barbaresco       0.32      0.16      0.21        38
                 Barolo 

In [123]:
cross = cross_val_score(pipelinep1, X_train3, y_train3, cv=5)
print(cross)
print(cross.mean())

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   27.5s
[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed:  1.5min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   28.0s
[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed:  1.5min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   28.1s
[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed:  1.5min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   28.0s
[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed:  1.6min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   32.5s
[Parallel(n_jobs=2)]: Done 149 out of 14

[0.6780628  0.6755348  0.67985807 0.68402575 0.682056  ]
0.6799074825393505


In [91]:
# CountVectorizer and Logisitic Regression with Lasso regularization
pipelinesp2 = Pipeline([
    ('vect', cvec),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr', verbose = 1, n_jobs = 2))
])
pipelinesp2.fit(X_train3, y_train3)
predictedp2 = pipelinesp2.predict(X_test3)
pipelinesp2.score(X_test3, y_test3)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


max_iter reached after 34 seconds


/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


max_iter reached after 37 seconds
max_iter reached after 31 seconds
max_iter reached after 71 seconds
max_iter reached after 23 seconds
max_iter reached after 32 seconds
max_iter reached after 111 seconds
max_iter reached after 37 seconds
max_iter reached after 34 seconds
max_iter reached after 33 seconds
max_iter reached after 50 seconds
max_iter reached after 35 seconds
max_iter reached after 30 seconds
max_iter reached after 29 seconds
max_iter reached after 32 seconds
max_iter reached after 59 seconds
max_iter reached after 39 seconds
max_iter reached after 32 seconds
max_iter reached after 122 seconds
max_iter reached after 73 seconds
max_iter reached after 47 seconds
max_iter reached after 102 seconds
max_iter reached after 30 seconds
max_iter reached after 39 seconds
max_iter reached after 39 seconds
max_iter reached after 73 seconds
max_iter reached after 84 seconds
max_iter reached after 34 seconds
max_iter reached after 329 seconds
max_iter reached after 44 seconds
max_iter r

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 20.9min


max_iter reached after 36 seconds
max_iter reached after 70 seconds
max_iter reached after 37 seconds
max_iter reached after 38 seconds
max_iter reached after 29 seconds
max_iter reached after 28 seconds
max_iter reached after 35 seconds
max_iter reached after 25 seconds
max_iter reached after 22 seconds
max_iter reached after 29 seconds
max_iter reached after 64 seconds
max_iter reached after 49 seconds
max_iter reached after 27 seconds
max_iter reached after 51 seconds
max_iter reached after 41 seconds
max_iter reached after 42 seconds
max_iter reached after 33 seconds
max_iter reached after 40 seconds
max_iter reached after 57 seconds
max_iter reached after 54 seconds
max_iter reached after 51 seconds
max_iter reached after 61 seconds
max_iter reached after 34 seconds
max_iter reached after 48 seconds
max_iter reached after 28 seconds
max_iter reached after 53 seconds
max_iter reached after 34 seconds
max_iter reached after 50 seconds
max_iter reached after 79 seconds
max_iter reach

[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed: 66.0min finished


0.6863602965733752

In [93]:
print(classification_report(y_test3, predictedp2))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

       Aconcagua Valley       0.00      0.00      0.00        26
      Adelaida District       0.11      0.04      0.06        26
               Alentejo       0.46      0.30      0.36       269
       Alexander Valley       0.00      0.00      0.00        24
                 Alsace       0.67      0.81      0.73       654
             Alto Adige       0.83      0.86      0.85        35
          Amador County       0.21      0.10      0.13        31
                America       0.80      0.14      0.24        29
              Andalucia       0.74      0.53      0.62        38
        Anderson Valley       0.11      0.06      0.08        31
        Australia Other       0.30      0.06      0.10        49
                  Baden       0.43      0.14      0.21        21
               Bairrada       0.94      0.41      0.57        37
             Barbaresco       0.30      0.08      0.12        38
                 Barolo 

In [94]:
joblib.dump(pipelinesp2, 'pipelinesp2_conca_province2')

['pipelinesp2_conca_province2']

In [95]:
pipe_reloaded2 = joblib.load('pipelinesp2_conca_province2')

In [37]:
#running
# TfidfVectorizer and Logisitic Regression with Ridge regularization
pipelines3 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(penalty = 'l2', solver='lbfgs', multi_class='ovr', verbose = 1, n_jobs = 2))
])
pipelines3.fit(X_train3, y_train3)
predicteds3 = pipelines3.predict(X_test3)
pipelines3.score(X_test3, y_test3)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   26.1s
[Parallel(n_jobs=2)]: Done 149 out of 149 | elapsed:  1.3min finished


0.6585398436978158

In [38]:
print(classification_report(y_test3, predicteds3))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

       Aconcagua Valley       0.00      0.00      0.00        26
      Adelaida District       0.00      0.00      0.00        26
               Alentejo       0.49      0.26      0.34       269
       Alexander Valley       0.00      0.00      0.00        24
                 Alsace       0.59      0.82      0.68       654
             Alto Adige       0.76      0.83      0.79        35
          Amador County       0.00      0.00      0.00        31
                America       0.00      0.00      0.00        29
              Andalucia       1.00      0.18      0.31        38
        Anderson Valley       0.33      0.06      0.11        31
        Australia Other       0.00      0.00      0.00        49
                  Baden       0.00      0.00      0.00        21
               Bairrada       0.91      0.27      0.42        37
             Barbaresco       0.00      0.00      0.00        38
                 Barolo 

In [141]:
# TfidfVectorizer and Logisitic Regression with Lasso regularization
pipelines4 = Pipeline([
    ('vect', vect),
    ('cls', LogisticRegression(penalty = 'l1', solver='saga', multi_class='ovr', ))
])
pipelines4.fit(X_train3, y_train3)
predicteds4 = pipelines4.predict(X_test3)
pipelines4.score(X_test3, y_test3)

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.6712644445928796

In [142]:
print(classification_report(y_test3, predicteds4))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

       Aconcagua Valley       0.00      0.00      0.00        26
      Adelaida District       0.00      0.00      0.00        26
               Alentejo       0.49      0.28      0.36       269
       Alexander Valley       0.00      0.00      0.00        24
                 Alsace       0.61      0.81      0.70       654
             Alto Adige       0.72      0.97      0.83        35
          Amador County       0.50      0.03      0.06        31
                America       0.67      0.07      0.12        29
              Andalucia       0.77      0.45      0.57        38
        Anderson Valley       0.25      0.13      0.17        31
        Australia Other       0.50      0.02      0.04        49
                  Baden       1.00      0.14      0.25        21
               Bairrada       0.89      0.43      0.58        37
             Barbaresco       0.38      0.08      0.13        38
                 Barolo 

### RANDOM FOREST CLASSIFIER, COUNTVECTORIZER AND TFIDFVECTORIZER
Logistic Regression performs better than Random Forest Classifier in this data set, with higher accuracy.

The best performed model was Random Forest with CountVectorizer.

All of the models have better scores than the baseline


In [130]:
cvecsr = CountVectorizer(strip_accents='unicode', stop_words="english", ngram_range=(1, 1))

In [144]:
# Random Forest with CountVectorizer
pipelines6  = Pipeline([
    ('vect', cvecsr),
    #('tfidf', TfidfTransformer()),
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2))
])
pipelines6.fit(X_train3, y_train3)
predicteds6 = pipelines6.predict(X_test3)
pipelines6.score(X_test3, y_test3)

0.5217420346002271

In [145]:
print(classification_report(y_test3, predicteds6))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

       Aconcagua Valley       1.00      0.15      0.27        26
      Adelaida District       0.00      0.00      0.00        26
               Alentejo       0.84      0.22      0.34       269
       Alexander Valley       0.00      0.00      0.00        24
                 Alsace       0.63      0.73      0.68       654
             Alto Adige       0.53      0.57      0.55        35
          Amador County       0.00      0.00      0.00        31
                America       1.00      0.10      0.19        29
              Andalucia       1.00      0.03      0.05        38
        Anderson Valley       0.00      0.00      0.00        31
        Australia Other       0.00      0.00      0.00        49
                  Baden       1.00      0.10      0.17        21
               Bairrada       1.00      0.08      0.15        37
             Barbaresco       0.06      0.03      0.04        38
                 Barolo 

In [133]:
# Random Forest with TfidfVectorizer and max_depth = 10
pipelines7 = Pipeline([
    ('vect', TfidfVectorizer(stop_words='english', sublinear_tf=True,
                             ngram_range = (1,1), strip_accents='unicode')),
    
    ('cls', RandomForestClassifier(n_estimators=200, n_jobs = 2, max_depth = 10 ))
])
pipelines7.fit(X_train3, y_train3)
predicteds7 = pipelines7.predict(X_test3)
pipelines7.score(X_test3, y_test3)

0.47790875551156303

In [134]:
print(classification_report(y_test3, predicteds7))

/Users/Mariagimenoperez/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                precision    recall  f1-score   support

     Argentina       0.00      0.00      0.00       919
     Australia       0.00      0.00      0.00       671
       Austria       0.00      0.00      0.00       791
        Brazil       0.00      0.00      0.00        11
      Bulgaria       0.00      0.00      0.00        39
        Canada       0.00      0.00      0.00        62
         Chile       0.00      0.00      0.00      1082
       Croatia       0.00      0.00      0.00        19
        Cyprus       0.00      0.00      0.00         2
Czech Republic       0.00      0.00      0.00         3
       England       0.00      0.00      0.00        27
        France       0.79      0.13      0.22      5586
       Georgia       0.00      0.00      0.00        26
       Germany       0.00      0.00      0.00       606
        Greece       0.00      0.00      0.00       111
       Hungary       0.00      0.00      0.00        34
         India       0.00      0.00      0.00  

### NAIVE BAYES, COUNTVECTORIZER AND TFIDFVECTORIZER
The best accuracy I got in MultinomialNB with CountVectorizer. MultinomialNB is one of the two classic Naive Bayes variants used in text classification.

Logistic Regression in all of models performs better than Native Bayes in this data set, with higher accuracy. But in this case, Naive Bayes performs better than Random Forest Classifier.

All of the models have better scores than the baseline


In [149]:
# MultinomialNB with CountVectorizer
pipelines8 = Pipeline([
    ('vect', cvec),
    ('cls', MultinomialNB())
])
pipelines8.fit(X_train3, y_train3)
predictes8 = pipelines8.predict(X_test3)
pipelines8.score(X_test3, y_test3)

0.578318081624474

In [153]:
# BernoulliNB with CountVectorizer
pipelines9 = Pipeline([
    ('vect', cvec),
    ('cls', BernoulliNB())
])
pipelines9.fit(X_train3, y_train3)
predictes9 = pipelines9.predict(X_test3)
pipelines9.score(X_test3, y_test3)

0.5359695411128181

In [156]:
# MultinomialNB with  TdidfVectorizer
pipelines10 = Pipeline([
    ('vect',vect) ,
    ('cls', MultinomialNB())
])
pipelines10.fit(X_train3, y_train3)
predictes10 = pipelines10.predict(X_test3)
pipelines10.score(X_test3, y_test3)

0.393226905350344